#ENGG*6600: Special Topics in Information Retrieval - Fall 2022
##Assignment 4: Relevance Models (Total : 100 points)

**Description**

This is a coding assignment where you will implement the RM1 model for Query Expansion. Basic proficiency in Python is recommended.  

**Instructions**

* To start working on the assignment, you would first need to save the notebook to your local Google Drive. For this purpose, you can click on *Copy to Drive* button. You can alternatively click the *Share* button located at the top right corner and click on *Copy Link* under *Get Link* to get a link and copy this notebook to your Google Drive.  

*   For questions with descriptive answers, please replace the text in the cell which states "Enter your answer here!" with your answer. If you are using mathematical notation in your answers, please define the variables.
*   You should implement all the functions yourself and should not use a library or tool for the computation.
*   For coding questions, you can add code where it says "enter code here" and execute the cell to print the output.
* To create the final pdf submission file, execute *Runtime->RunAll* from the menu to re-execute all the cells and then generate a PDF using *File->Print->Save as PDF*. Make sure that the generated PDF contains all the codes and printed outputs before submission.
To create the final python submission file, click on File->Download .py.


**Submission Details**

* Due data: Nov. 11, 2022 at 11:59 PM (EST).
* The final PDF and python file must be uploaded on GourseLink.
* After copying this notebook to your Google Drive, please paste a link to it below. Use the same process given above to generate a link. ***You will not recieve any credit if you don't paste the link!*** Make sure we can access the file.
***LINK: *https://colab.research.google.com/drive/1Imx2uUD3il2VSk8JJhGQgwPYPc7szSVr?usp=sharing***

**Academic Honesty**

Please follow the guidelines under the *Collaboration and Help* section in the first lecture.

# Download input files and code

Please execute the cell below to download the input files.

In [ ]:

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


import os
import zipfile

download = drive.CreateFile({'id': '1CLVPQ0HADjmE0cPthdZ628aIYc8yvMdq'})
download.GetContentFile('HW04.zip')

with zipfile.ZipFile('HW04.zip', 'r') as zip_file:
    zip_file.extractall('./')
os.remove('HW04.zip')
# We will use hw1 as our working directory
os.chdir('HW04')

#Setting the input files
queries_file = "queries_tok_clean_kstem"
col = "antique-collection.tok.clean_kstem"
query_pass = "query_pass"

# 1 : Initial Data Setup (20 points)

We use files from the ANTIQUE  [https://arxiv.org/pdf/1905.08957.pdf] dataset for this assignment. As described in the previous assignments, this is a passage retrieval dataset.

The description of the input files provided for this assignment is given below.

**Query File**

We randomly sampled a set of 5 queries from the test set of the ANTIQUE dataset. Each row of the input file contains the following information:

*query_id query_text*

The id and text information is tab separated. query_id is a unique identifier for a query and query text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer.  

**Collection file**

Each row of the file consists of the following information:

*passage_id  passage_text*

The id and text information is tab separated. The passage text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer (same as queries). The terms in the passage text can be accessed by splitting the text based on space.

**Query Feedback Passages**

This file consists of queries and 10 feedback passages corresponding to each query.  Each row of the file consists of the following information:

*query_id  passage_id1 passage_id2 ......passage_id10*

The entries are space separated. The first column is the query_id followed by the passage_ids.

In this section, you have to implement the following:

* Load the queries from the query file into a datastructure
* Load the query feedback passages into a datastructure.

You can use any additional datastructures than the suggested ones for your implementation.




In [ ]:

'''
This function is used to load query file information into datastructure(s).
Return Variables:
queries - mapping from queryid to querytext
'''

def loadQueries(queries_file):
  #enter your code here
  queries={}
  for line in open(queries_file,encoding='utf8'):
        qid,qtext=line.strip().split('\t')
        queries[qid]=qtext

  return queries


'''
This function is used to load feedback passages corresponding to the queries into a datastructure.
The file format is given below:
"query_id passage_id1 passage_id2 .....   passage_id10"
The entries are space separated.

Return Variables:
num_queries - number of queries in the file
feedback_pass - mapping from queryid to list of feedback passages
'''


def loadFeedbackPass(query_pass):
     #enter your code here
     feedback_pass={}
     for line in open("query_pass",encoding='utf8'):
          x=line.strip().split(' ')
          feedback_pass[x[0]]=x[1:]


     return feedback_pass



queries = loadQueries(queries_file)
feedback_pass = loadFeedbackPass(query_pass)


print ('Total Num of queries in the query file  : {0}'.format(len(queries)))
print ('Total Num of queries in the feedback file  : {0}'.format(len(feedback_pass)))



Total Num of queries in the query file  : 5
Total Num of queries in the feedback file  : 5



In the cell below, an inverted index with count has been created in memory. Please run the cell and use the variables for implementing the relevance feedback models.

In [ ]:
'''
Store the feedback docids
'''

def storeFeedbackPass(query_pass):
    feedback_pass_ids = {}
    for line in open(query_pass):
      row = line.strip().split(' ')
      for p in row[1:]:
        if p not in feedback_pass_ids:
          feedback_pass_ids[p]=0
        feedback_pass_ids[p]+=1
    return feedback_pass_ids

feedback_pass_ids = storeFeedbackPass(query_pass)


'''
An inverted index with count information.
'''
class indexCount:
   pcount = 0
   ctf = {}
   sumdl = 0
   avgdl = 0
   doclen = {}
   index = {}
   probctf = {}
   feedback_pass_contents = {}

   def __init__(self, col):
     self.col = col


   def create_index(self):
     for line in open(self.col):
       pid,ptext = line.strip().split('\t')
       self.pcount+=1

       if pid not in self.doclen:
           self.doclen[pid]=0
       pfreq = {}
       for term in ptext.split(' '):
           self.doclen[pid]+=1

           if term not in pfreq:
              pfreq[term]=0
           pfreq[term]+=1


       for k,v in pfreq.items():
        if k not in self.index:
          self.index[k]={}

        self.index[k][pid]=v

       if pid in feedback_pass_ids:
          self.feedback_pass_contents[pid]=ptext


buildIndex = indexCount(col)
buildIndex.create_index()


'''
inverted index with count: nested dict with term and passage_id as key"
Example - {'the': {'2020338_0:11', '3174498_1:4'}}
'''
index = buildIndex.index

#total number of passages in the collection
num_passages = buildIndex.pcount

#dict with passageId as key and number of terms in the passage as value
doclen = buildIndex.doclen

#dict with passage id as key and passage text as value for feedback passages
feedback_pass_contents = buildIndex.feedback_pass_contents

print('Total number of passages in the collection :{0}'.format(num_passages))
print('Total number of feedback passages :{0}'.format(len(feedback_pass_contents)))


Total number of passages in the collection :403492
Total number of feedback passages :50


# 2 : Query Language Model (30 points)

In the cell below, calculate Maximum Likelihood Estimates for the Query Language Model, as follows:

$P_{MLE}(t|q)$ = $\frac{count(t,q)}{|q|}$

$count(t,q)$ - Number of times term $t$ appears in query $q$

$|q|$ - Number of tokens in query $q$

Calculate the estimates for all queries.


In [ ]:
'''
In this function implement mle estimates for all queries.
Return Variables:
  mle_queries - mapping from queryid to mle estimates. The mle estimates for each query is a dict from each query word to its MLE probability.
'''

def calcMleQueries(queries):
  #enter your code here
  mle_queries={}
  for q in queries:
        num_tokens =len(queries.get(q).split())
        query_vocab = []
        for word in queries.get(q).split():
            if word not in query_vocab:
                query_vocab.append(word)

        query_wc = {}
        for word in query_vocab:
            query_wc[word] = queries.get(q).split().count(word)

        mle={}
        for word in query_vocab:
            p=query_wc[word]/num_tokens
            mle[word]=p
        mle_queries[q]=mle

  return mle_queries

mle_queries = calcMleQueries(queries)

# Hint: the MLE estimates are in the interval [0.10,0.20]
print('MLE estimates for qid 3396066 :{0}'.format(mle_queries['3396066']))


MLE estimates for qid 3396066 :{'why': 0.16666666666666666, 'do': 0.16666666666666666, 'airplane': 0.16666666666666666, 'fly': 0.16666666666666666, 'so': 0.16666666666666666, 'high': 0.16666666666666666}


# 3 : Relevance Model 1 (RM1) (50 points)

In the cell below, implement the RM1 feedback model, as follows.

$$P_{RM1}(t|\theta_{F}) \propto \sum_{p \in F} (p(t|\theta_{p}) \prod_{w \in q} p(w|\theta_{p}))$$

$p(t|\theta_{p}) = \frac{count(t,p) + \delta}{|p| + \delta|V|}$ ($p(w|\theta_{p})$ is computed similarly)

$\delta = 0.1$

$|V|$: vocabulary size (number of unique words in the vocabulary)

$count(t,p)$ - Number of times term $t$ occurs in passage $p$

$|p|$ - Number of tokens in passage $p$

$F$: Feedback passages

For every query, this has to be computed for all unique terms present in feedback passages.

**Note:** Once you compute the weights for each term, you should normalize them by dividing by their sum. In other words, the RM1 probabilities for each query should sum to 1.


In [ ]:
query_vocab_dict={}
for q in queries:
        num_tokens =len(queries.get(q).split())
        query_vocab = []
        for word in queries.get(q).split():
            if word not in query_vocab:
                query_vocab.append(word)

        query_wc = {}
        for word in query_vocab:
            query_wc[word] = queries.get(q).split().count(word)
        query_vocab_dict[q]=query_wc


In [ ]:
vocab_qid={}
for qid in feedback_pass:
    l=feedback_pass[qid]
    feedback_vocab=[]
    for i in l:
        for word in feedback_pass_contents[i].split():
            if word not in feedback_vocab:
                feedback_vocab.append(word)

    vocab_qid[qid]=feedback_vocab

In [ ]:
def normalize(prm1):
    total=0
    for i in prm1:
        total=total+prm1[i]
    for i in prm1:
        prm1[i]=prm1[i]/total

    return prm1


In [ ]:
'''
In this function implement RM1 and return words and their probabilities.
Return Variables:
rm1 - mapping from queryid to RM1 probabilities.
The RM1 probabilities for each query is a dict from the words that appear in the feedback passages to their RM1 probability.
'''
import collections

def calcRM1(index,queries,doclen,feedback_pass,feedback_pass_contents):
    #enter your code here

    rm1={}
    for qid in feedback_pass:
        prm1={}
        #first setting it to zero
        for w in vocab_qid[qid]:
            prm1[w]=0

        for i in feedback_pass[qid]:
            qidx=1
            for word in query_vocab_dict[qid]:
                qwcount=query_vocab_dict[qid][word]
                #print(len(feedback_pass_contents[fp]),len(vocab_qid[qid]))
                qidx=qidx*(qwcount+0.1)/(len(feedback_pass_contents[i])+int(0.1*len(index)))


            for word in feedback_pass_contents[i].split():
                pwcount=feedback_pass_contents[i].count(word)
                x=(pwcount+0.1)/(len(feedback_pass_contents[i])+int(0.1*len(index)))
                #print(word,x)
                r=x*(qidx)
                prm1[word]=r

        pnrm=normalize(prm1)
        rm1[qid]=pnrm

    return rm1

rm1 = calcRM1(index,queries,doclen,feedback_pass,feedback_pass_contents)

'''
Print out top 20 terms and corresponding probabilities
this assumes that the rm1 variable returned is a dict with queryid as key and dict consisting of term and probability values as the value.
You can alter this based on your implementation.
'''

rm1_scores = {}
rm1_final = {}
for k,v in rm1.items():
    if k not in rm1_final:
      rm1_final[k]=[]
    if k not in rm1_scores:
      rm1_scores[k]={}
    sorted_p = sorted(v.items(), key=lambda x: x[1], reverse=True)
    sorted_dict = collections.OrderedDict(sorted_p)
    for t,s in sorted_dict.items():
        rm1_final[k].append(t+":"+str(s))
        rm1_scores[k][t]=s

''' Hint: The probability value or the top (1st) term is in the range [0.081,0.083]
          The probability value for the 20th term is in the range [0.0076,0.0078]'''
print('Top 20 Feedback terms and their RM1 probabilities for qid 3396066 :{0}'.format(rm1_final['3396066'][:20]))



Top 20 Feedback terms and their RM1 probabilities for qid 3396066 :['i:0.05189559068351684', 't:0.045073874202802476', 'a:0.03832965362382953', 'u:0.030991937693658916', 's:0.026645732805317773', 'n:0.02146910217639719', 'an:0.01866466148141698', 'the:0.012201992135104929', 'ear:0.01160929116198101', 'm:0.010603420315491136', 'd:0.010603420315491136', 'helicopter:0.006954797369551261', 'to:0.0061614019692114', 'or:0.005836715446078849', 'cause:0.005836715446078849', 'pressure:0.005836715446078849', 'on:0.005732587492126997', 'balls:0.0050637836726273', 'in:0.004953283936032693', 'cloud:0.00491255895608429']
